In [34]:
from pathlib import Path
import pandas as pd
import tidytcells as tt
from tqdm import tqdm

In [2]:
raw_dir = Path('/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/raw/mira')
preprocessed_dir = Path('/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/data/preprocessed/mira')

## Load data

In [3]:
ci = pd.read_csv(raw_dir/"peptide-detail-ci.csv")
cii = pd.read_csv(raw_dir/"peptide-detail-cii.csv")

### Data summary

In [27]:
ci_size = len(ci)
cii_size = len(cii)

ci_groups = ci["Amino Acids"].nunique()
cii_groups = cii["Amino Acids"].nunique()

print(
    f"""
    Data sizes:
    ci: {ci_size}
    cii:{cii_size}

    Group sizes:
    ci: {ci_groups}
    cii:{cii_groups}
    """
)


    Data sizes:
    ci: 154320
    cii:6809

    Group sizes:
    ci: 269
    cii:56
    


## Reformat data

In [28]:
ci.head()

,TCR BioIdentity,TCR Nucleotide Sequence,Experiment,ORF Coverage,Amino Acids,Start Index in Genome,End Index in Genome
0,CASSAQGTGDRGYTF+TCRBV27-01+TCRBJ01-02,GAGTCGCCCAGCCCCAACCAGACCTCTCTGTACTTCTGTGCCAGCA...,eAV93,"ORF1ab,surface glycoprotein","ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",533,24073
1,CASSLVATGNTGELFF+TCRBV07-09+TCRBJ02-02,CGCACAGAGCAGGGGGACTCGGCCATGTATCTCTGTGCCAGCAGCT...,eOX56,"ORF1ab,surface glycoprotein","ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",533,24073
2,CASSKGTVSGLSG+TCRBV21-01+TCRBJ02-07,GAGATCCAGTCCACGGAGTCAGGGGACACAGCACTGTATTTCTGTG...,eAV93,"ORF1ab,surface glycoprotein","ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",533,24073
3,CALKVGADTQYF+TCRBV30-01+TCRBJ02-03,CTGAGTTCTAAGAAGCTCCTTCTCAGTGACTCTGGCTTCTATCTCT...,eQD124,"ORF1ab,surface glycoprotein","ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",533,24073
4,CASSLWASGRGGTGELFF+TCRBV27-01+TCRBJ02-02,AGCCCCAACCAGACCTCTCTGTACTTCTGTGCCAGCAGTTTATGGG...,eAV93,"ORF1ab,surface glycoprotein","ADAGFIKQY,AELEGIQY,LADAGFIKQY,TLADAGFIK",533,24073


In [41]:
ci[["CDR3B", "TRBV", "TRBJ"]] = ci.apply(lambda x: x["TCR BioIdentity"].split("+"), result_type="expand", axis=1)

In [42]:
ci["TRBV"] = ci["TRBV"].map(lambda x: tt.tcr.standardise(x, enforce_functional=False))

/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TCRBV27-01" for species homosapiens. Attempted fix "TRBV27-1" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TCRBV30-01" for species homosapiens. Attempted fix "TRBV30-1" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarning: Failed to standardise: "TCRBV12-X" for species homosapiens. Attempted fix "TRBV12-X" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tcr_embedder/.venv/lib64/python3.8/site-packages/tidytcells/_utils/warnings.py:10: UserWarnin

In [44]:
ci["TRBV"].notna().sum()

70101